In [ ]:
!pip install requests
import requests
import os
import json


In [ ]:
#DEFAULT VARIABLES 
#do not change unless you know what you are doing
api_key = 'HDEV-0465ed95-8059-4c6d-b08f-40831d5c957d'
mode = "competitive" #no caps

allMaps = {"/Game/Maps/Ascent/Ascent": "Ascent", 
"/Game/Maps/Bonsai/Bonsai": "Split",
"/Game/Maps/Canyon/Canyon": "Fracture",
"/Game/Maps/Duality/Duality": "Bind",
"/Game/Maps/Foxtrot/Foxtrot": "Breeze",
"/Game/Maps/Jam/Jam": "Lotus",
"/Game/Maps/Juliett/Juliett": "Sunset",
"/Game/Maps/Pitt/Pitt": "Pearl",
"/Game/Maps/Port/Port": "Icebox",
"/Game/Maps/Triad/Triad": "Haven",
"/Game/Maps/HURM/HURM_Alley/HURM_Alley": "District",
"/Game/Maps/HURM/HURM_Bowl/HURM_Bowl": "Kasbah",
"/Game/Maps/HURM/HURM_Helix/HURM_Helix": "Drift",
"/Game/Maps/HURM/HURM_Yard/HURM_Yard": "Piazza",
"/Game/Maps/Poveglia/Range": "The Range"
}

allGuns ={

    #WeaponID conversions
}

allArmor = {
    #ArmorID conversions
    
}

DBUG = False
if DBUG:
    dbug_print = print
else:
    dbug_print = lambda *args, **kwargs: None


import time

#USELESS
# from collections import deque

# all_requests = deque()

# def check_time():
    
#     global all_requests

#     print(f"\t\t\trequests made in the last min: [{len(all_requests)}]")
#     if(len(all_requests) >= 30):
#         timediff = time.time() - all_requests.popleft()
#         print(f"\t\t\t\ttime elapsed from oldest request: [{timediff}]")
#         if(timediff < 60):
#             print(f"\t\t\t\tsleeping for [{60-timediff}] sec")
#             time.sleep(60-timediff)
        
#         check_time()
# def put_time():
#     global all_requests

#     print("\t\t\trequest made.")
#     all_requests.append(time.time())


#ensures that we dont exceed the max requests. write this after every API request!!!!!
def limit_check(response):
    print("\t\t\trequest made.")
    print(f"\t\t\t\tAlleged requests made: [{response.headers['x-ratelimit-limit']}]")
    if(int(response.headers['x-ratelimit-limit']) >= 30):
        print(f"\t\t\t\tsleeping for [{response.headers['x-ratelimit-reset']}]")
        time.sleep(int(response.headers['x-ratelimit-reset']))
        
    


In [ ]:
#Accesses API
#returns puuid of player
def get_puuid(name: str, tag: str, api_key: str):

    url = f"https://api.henrikdev.xyz/valorant/v1/account/{name}/{tag}"
    headers = {'Authorization': f'{api_key}'}


    response = requests.get(url, headers=headers)
    limit_check(response)

  
 
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch puuid: {response.status_code}")
        error = response.json()['errors'][0]['message']
        details= response.json()['errors'][0]['details']
        code = response.json()['status']
        if details == "null":
            details = ""     
        else:
            details = f"Details: {details}"
        response_message = f"{error} Code:{code}....{details}"
        print(response_message, "\nEXITING PROGRAM")
        quit()

#Accesses API
#returns the player's raw data match history
#format is here: https://valapidocs.techchrism.me/endpoint/match-history 
def get_raw(puuid: str,api_key: str,queue: str, startIndex: int, endIndex: int):
    url = f"https://api.henrikdev.xyz/valorant/v1/raw"
    headers = {
        'Authorization': f'{api_key}',
    }
    body = {
        "type": "matchhistory",
        "value": puuid,
        "region": "na",
        "queries": f"?startIndex={startIndex}&endIndex={endIndex}&queue={queue}"
    }
    
    
    response = requests.post(url, headers=headers, json=body)
    limit_check(response)

 
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch raw_data: {response.status_code}")
        error = response.json()['errors'][0]['message']
        details= response.json()['errors'][0]['details']
        code = response.json()['status']
        if details == "null":
            details = ""     
        else:
            details = f"Details: {details}"
        response_message = f"{error} Code:{code}....{details}"
        print(response_message, "\nEXITING PROGRAM")
        quit()

#Accesses API
#returns match details of matchID
#format is here: https://valapidocs.techchrism.me/endpoint/match-details 
def get_match_details(matchID: str,api_key: str):
    url = f"https://api.henrikdev.xyz/valorant/v1/raw"
    headers = {
        'Authorization': f'{api_key}',
    }
    body = {
        "type": "matchdetails",
        "value": matchID,
        "region": "na",
    }
    
    

    response = requests.post(url, headers=headers, json=body)
    limit_check(response)
 
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch raw_data: {response.status_code}")
        error = response.json()['errors'][0]['message']
        details= response.json()['errors'][0]['details']
        code = response.json()['status']
        if details == "null":
            details = ""     
        else:
            details = f"Details: {details}"
        response_message = f"{error} Code:{code}....{details}"
        print(response_message, "\nEXITING PROGRAM")
        quit()

#Does not access API
#auxillary function for get_all_location_data
#given match details (format: https://valapidocs.techchrism.me/endpoint/match-details)
#mutates output_dict by placing new data inside
def analyze_match(curr_match: dict, puuid: str, output_dict: dict):
    #if map no exist, make it exist
    

    map_id = curr_match['matchInfo']['mapId']
    map_name = allMaps[map_id]


    print(f"----CURRENT MAP: [{map_name}]----")

    if map_name not in output_dict:
        output_dict[map_name] = {

            'All_Kill_Times' : [0]*101, #indexes 0 thru 100 = 101 elements (these are % numbers)
            'All_Death_Times': [0]*101,

            "Attack": {
                #make these all linkedlists which apparently is using deque()
                "kill_info": [[] for _ in range(160)],
                "death_info": [[] for _ in range(160)],
                #WE ARE DITCHING THE PLANT INFO!, making a separate dictionary that gets ALL plant data and defuse data(not just specific player)
            },
            "Defense": {
                "kill_info": [[] for _ in range(160)],
                "death_info": [[] for _ in range(160)]
            }
        }

    #determine what team our player is on
    #so that we can tell if they are on attack or defense on a given round
    player_team = None
    for player in curr_match['players']:
        if player['subject'] == puuid:
            player_team = player['teamId']
            break
    if player_team == "Red":
        team_side = "Attack"
    elif player_team == "Blue":
        team_side = "Defense"

    round_count = 0
    kill_array_index = 0
    for round_result in curr_match['roundResults']:
        if kill_array_index < len(curr_match['kills']): #as long as there are still kills to be checked, we continue
            dbug_print("\n")
            
            round_count += 1
            dbug_print('currently examining new round [', round_count, "]")
            dbug_print('  player is on ', team_side)

            if round_count > 24 or ( round_count % 12 == 1 and round_count > 1):  
                if team_side == "Attack":
                    team_side = "Defense"
                else:
                    team_side = "Attack"
                dbug_print('--------------SWITCHING SIDES: player is now on ', team_side)


            #get specified player's stats
            for player_stat in round_result['playerStats']:
                if player_stat['subject'] == puuid:
                    dbug_print("\tplayer stats found")
                    curr_round_stats = player_stat
            
            #location data will need these things
            loadout_value = curr_round_stats['economy']['loadoutValue']
            weapon = curr_round_stats['economy']['weapon']
            armor  = curr_round_stats['economy']['armor']

            #determine where to put the next entry in the array by loadout value
            array_spot = (int)(loadout_value/50)
            dbug_print("\tloadout value[", loadout_value, "]->array spot[", array_spot, "]")
        #calculate round length

            #calculate start time 
            round_start_time = curr_match['kills'][kill_array_index]['gameTime'] - curr_match['kills'][kill_array_index]['roundTime']
            
            #calculate start time of next round, if there is no next round, time = -1
            next_round_start_time = -1
            for kill_info in curr_match['kills']:
                if kill_info['round'] == round_count + 1:
                    next_round_start_time = kill_info['gameTime'] - kill_info['roundTime']
                    break
        

            #subtract the buy phase time
            buy_phase = 0
            if(round_count == 1 or round_count == 13 or round_count == 25):
                buy_phase = 45000
            else:
                buy_phase = 30000

            if(next_round_start_time == -1):
                round_length = curr_match['matchInfo']['gameLengthMillis'] - round_start_time #might be wrong because gameLengthMillis might not start from the start of round 1
            else:
                round_length = next_round_start_time - round_start_time - buy_phase
        #round length calculated
        
            dbug_print('\t\tround start(ms)  = ', round_start_time)
            dbug_print('\t\tnext round start(ms)  = ', next_round_start_time)
            dbug_print('\t\tbuy phase(ms)    =', buy_phase)
            dbug_print("\t\tround length(ms) = ", round_length)



            
            #loop through all kills in the current round
            while(kill_array_index < len(curr_match['kills']) and curr_match['kills'][kill_array_index]['round']+1 == round_count):
                dbug_print('\t\t\tcurrently examining kills on round ', curr_match['kills'][kill_array_index]['round']+1, 'should match ', round_count)
                curr_kill_info = curr_match['kills'][kill_array_index]

                if(curr_kill_info['victim'] == puuid):#our player died!

                    death_time_percentage = (int)(round(curr_kill_info['roundTime']/round_length, 2)*100)
                    dbug_print("\t\t\t\tdeath_time[0,100]: ", death_time_percentage, "%")#DBUG

                    dbug_print('\t\t\t\tINSERTING death INTO SLOT: [', array_spot, "]")
                    output_dict[map_name][team_side]['death_info'][array_spot].append({

                        'Location': {'x': curr_kill_info['victimLocation']['x'], 'y':curr_kill_info['victimLocation']['y']},
                        'Armor' : armor,
                        
                    })

                    #add to the death time
                    output_dict[map_name]['All_Death_Times'][death_time_percentage]+=1
                
                if(curr_kill_info['killer'] == puuid):#our player got a kill!
                    
                    kill_time_percentage = (int)(round(curr_kill_info['roundTime']/round_length,2)*100)
                    dbug_print("\t\t\t\tkill_time[0,100]: ", kill_time_percentage, "%")#DBUG

                    #add kill info (a bit more complicated)
                    for player_loc in curr_kill_info['playerLocations']:

                        if player_loc['subject'] == puuid:

                            dbug_print('\t\t\t\tINSERTING kill INTO SLOT: [', array_spot, "]")
                            output_dict[map_name][team_side]['kill_info'][array_spot].append({
                                'Location': {'x': player_loc['location']['x'], 'y': player_loc['location']['y']},
                                'ViewRadians': player_loc['viewRadians'],
                                'Weapon': weapon
                            })

                    #add to the kill time
                    output_dict[map_name]['All_Kill_Times'][kill_time_percentage]+=1
                


                kill_array_index += 1
    return output_dict


#Accesses API(indirectly)
#returns specified(puuid) player's data 
#data structure image link: [to be added]
def get_all_location_data(puuid: str, all_match_ids: []):
    to_return_dict = {}

    for index, match_id in enumerate(all_match_ids):
        curr_match = get_match_details(match_id, api_key)

        print("\nExamining Match [", index+1, "]")

        to_return_dict = analyze_match(curr_match, puuid, to_return_dict)


    return to_return_dict


In [9]:
#CREATE AN ARRAY OF ALL OF THE MATCH IDS THAT I HAVE ACCESS TO (from the API)


name = "GEA joli" #change these :)
tag = "brrr"



puuid: str = get_puuid(name, tag, api_key)['data']['puuid']
match_history: dict = get_raw(puuid, api_key, mode ,0 ,25)
total_elements: int = match_history['Total']
history = []

history.extend([match['MatchID'] for match in match_history['History']])

if total_elements > 25:#matchhistory: limit = 25 at a time
    for i in range(25, total_elements, 25):
        next_batch = get_raw(puuid, api_key, mode, i, i+25)
        history.extend([match['MatchID'] for match in next_batch['History']])

print("history[]")
print(f"   Type: {type(history)} ")
print(f'   Length: [{len(history)}]')






			request made.
				Alleged requests made: [1]
			request made.
				Alleged requests made: [2]
			request made.
				Alleged requests made: [3]
			request made.
				Alleged requests made: [4]
			request made.
				Alleged requests made: [5]
history[]
	Type: <class 'list'> 
	Length: [87]


In [ ]:
#Warning: this may take a long time (about 2 seconds per item in history[])
#history[] is full of matchIDs, so feel free to shorten it or do whatever
#87 elements took 3min 13sec
test_allData = get_all_location_data(puuid, history) 


#I will change up the get_all_location_data function for more customizability, and create more data structures
#instead of having return values it will just mutate multiple dictionaries probably
#   kill + death distance
#   fix kill and death time!!!! and put in separate map
#   maybe more cool things